# Person

> Defines a person class to be used when modelling the outbreak of a virus among people

In [1]:
#| default_exp Person

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#%pip install networkx[default]

In [4]:
#| export
#%pip install PyQt5

In [5]:
#| export
#%pip install PySide2

In [6]:
#| export
import matplotlib
import os

In [7]:
#| export
matplotlib.use('TkAgg')
if os.getcwd().split('\\')[-1] == 'nbs':
    os.chdir('../')

In [8]:
#| export
import numpy as np
import scipy
import AgentBasedModelling.pycxsimulator
from pylab import *

### Initialize State

In [ ]:
#| export
import json
import random

class Person():
    """
    This class set ups the initial values for agents in our system.
    """
    global PersonID
    PersonID = 1
    def __init__(self, sex_probability:dict, age_probability:dict, fertility_rate:dict, mortality_rate:dict) -> None:
        global PersonID
        self.id = PersonID
        # Assigns a random age and sex based on the Irish population census data
        self.age = return_random_choice(age_probability) 
        self.sex = return_random_choice(sex_probability)
        # Assigns the fertility_rate, mortality_rate based on the average for a person that age.
        self.fertility_rate = 0 if self.sex == "Male" else fertility_rate.get(self.age, 0)
        self.mortality_rate = mortality_rate.get(self.age, 0)

        # Covid Variables
        self.vaccinated = 0 # unvaccinated, vaccinated, one booster = 0, 1, 2
        self.infected = 0 # uninfected, is infected, was infected = 0, 1, 2
        
        # Position of Person in grid
        self.x = random.uniform(0, 999)
        self.y = random.uniform(0, 999)

        # Other variables ? mobility?
        # self.mobility = could be a random int or a fixed variable

        
        
        PersonID += 1
    
    def __str__(self):
        return f"Person {self.id} is a {self.age} year old {self.sex}.\n\
They have the fertility_rate {self.fertility_rate} and the mortality_rate {self.mortality_rate}\n"

In [9]:
infectionDistanceSquared = infectionDistance() ** 2

# agents state variables
uninfected = 0
infected = 1
recovered = 2

# to be removed from the simulation
toBeRemoved = -1

def initialize():
    global time, agents
    
    time = 0
    
    # initialize agents
    agents = []
    for i in range(populationSize()):
        if i < numInfected():
            state = infected
        else:
            state = uninfected
        agents.append([uniform(0, width()), uniform(0, height()), state, 0])


### Observe

In [10]:
#| export
def observe():
    cla()
    uninfectedX = [ag[0] for ag in agents if ag[2] == uninfected]
    uninfectedY = [ag[1] for ag in agents if ag[2] == uninfected]
    infectedX = [ag[0] for ag in agents if ag[2] == infected]
    infectedY = [ag[1] for ag in agents if ag[2] == infected]
    recoveredX = [ag[0] for ag in agents if ag[2] == recovered]
    recoveredY = [ag[1] for ag in agents if ag[2] == recovered]
    scatter(uninfectedX, uninfectedY, color = 'cyan')
    scatter(infectedX, infectedY, color = 'red')
    scatter(recoveredX, recoveredY, color = 'green')
    axis('scaled')
    axis([0, width(), 0, height()])
    title('t = ' + str(time))

### Clip

In [11]:
#| export
def clip(a, amin, amax):
    if a < amin: return amin
    elif a > amax: return amax
    else: return a

### Update State

In [12]:
#| export
def update():
    global time, agents

    time += 1

    # simulate random motion
    for ag in agents:
        ag[0] += normal(0, 1)
        ag[1] += normal(0, 1)
        ag[0] = clip(ag[0], 0, width())
        ag[1] = clip(ag[1], 0, height())

    # detect infection and change state
    x, y = 0, 1
    for i in range(len(agents)):
        if agents[i][2] == infected:
            for j in range(len(agents)):
                if agents[j][2] == uninfected and (agents[i][x]-agents[j][x])**2 + (agents[i][y]-agents[j][y])**2 < infectionDistanceSquared:
                    if random() < infectionRate():
                        agents[j][2] = infected
                        agents[j][3] = time

        if agents[i][2] == infected and random() < recoveryRate():
            agents[i][2] = recovered

    # remove "toBeRemoved" agents
    while toBeRemoved in [ag[2] for ag in agents]:
        for ag in agents:
            if ag[2] == toBeRemoved:
                agents.remove(ag)
                break

### Running the Simulation

In [13]:
#| export
parameters = [
    # Simulation parameters
    height, width, populationSize, numInfected,
    infectionDistance, infectionRate, recoveryRate
]
pycxsimulator.GUI(parameterSetters=parameters).start(func=[initialize, observe, update])

# Implementation Using Mesa

In [3]:
%pip install -Uqq mesa

Note: you may need to restart the kernel to use updated packages.


In [4]:
#| export
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import random

In [ ]:
#| export
class Person(Agent):
    '''A person who can become infected or immune'''
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.infected = False
        self.immune = False

    def move(self):
        '''Move the person to a neighboring cell'''
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False
        )
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def infect(self):
        '''Infect the person'''
        self.infected = True

    def step(self):
        '''Advance the person by one step in time'''
        self.move()
        if self.infected:
            self.immune = True
            self.infected = False

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()